# Assignment 11a, GR10NR11

In this assigment, the c-curves were established for different fractions of the alloy AA6082 along with the T-t curves for different cooling conditions. 

In [31]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import *
from IPython.display import clear_output
import pandas as pd
import random as rd
import scipy

newparams = {'figure.figsize': (15, 9), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral', 'figure.dpi': 200}
plt.rcParams.update(newparams)

In [32]:
#constants for c-curve
Teq = 500+273 #Kelvin
A_0 = 420 #J/mol
Qd = 130000 #J/mol
Tr = 300+273 #Kelvin
tr = 10 #s
n = 0.75
Xc = 0.05
R = 8.314 #J/Kmol

X = np.arange(0.05, 1, 0.1)
T1 = np.linspace(500+273, 270+273, 1000)

## C-curve

To construct the C-curves, the following equations were used:

$$ t^{*} = t_{r}  exp(\frac{A_{0}}{R}(\frac{T_{eq}^{2}}{T(T_{eq}-T)^{2}}-\frac{T_{eq}^{2}}{T_{r}(T_{eq}-T_{r})^{2}}+\frac{Q_{d}}{R}(\frac{1}{T}-\frac{1}{T_{r}})) $$

where the r refers to a given reference condition, A0 is a parameter that reflects the barrier against nucleation in the material, $T_{eq}$ is an equilibrium parameter for the phase transformation, and $Q_{d}$ is the activation energy for diffusion in the material. R is the gas constant, T is the temperature and t is the time.

$$ t = t^{*}(\frac{ln(1-X)}{ln(1-X_{C}})^{\frac{1}{n}} $$

In [33]:
def t_star(T):
    ts = tr*np.exp((A_0/R)*(((Teq**2)/(T*(Teq-T)**2))-(Teq**2/(Tr*(Teq-Tr)**2)))+(Qd/R)*((1/T)-(1/Tr)))
    return ts


def t(frac):
    t = np.zeros(len(T1))
    for i in range(len(T1)):
        ts = t_star(T1[i])
        t[i] = ts*(np.log(1-frac)/np.log(1-Xc))**(1/n)
    return t


def Cplot():
    colors = ['crimson', 'red', 'orange', 'chartreuse', 'limegreen', 'turquoise', 'cyan', 'blue','purple', 'hotpink']
    col = 0
    for i in range(len(X)):
        t1 = t(X[i])
        plt.plot(t1, T1, label='X =' + str(np.round(X[i], 2)), color=colors[col])
        col += 1
    plt.grid()
    plt.title('C-curve for AA6082')
    plt.xscale('log')
    plt.xlabel('log t')
    plt.xlim(10**(-1), 10**7)
    plt.ylabel('Temperature, C')
    plt.legend(title='Fraction transformed:')
    plt.show()


## Jominy-stab

The following equation was used to construct the T-t profile for jominy test:

$$ \frac{T-T_{0}}{T_{i}-T_{0}} = erf (\frac{x}{2\sqrt{at}}) $$

where Ti is the initial temperature of the stick, T0 is the temperature at the end of the stick where it is quenched and a is the thermal diffusivity of the material.

In [34]:
#constants
Ti = 550+273 #Celcius
T0 = 20+273 #Celcius
a = 62 #mm2/s

In [106]:
x = np.arange(10, 101, 10)
tj = np.logspace(-2, 6, 100)

def jominy(pos):
    T = np.zeros(len(tj))
    for i in range(len(T)):
        T2 = T0 + (Ti-T0)*scipy.special.erf(pos/(2*np.sqrt(a*tj[i])))
        if T2 > 308:
            T[i] = T2
    return np.trim_zeros(T)

def jplot(valg):
    colors = ['darkblue','royalblue','teal', 'limegreen','greenyellow', 'gold', 'darkorange','coral', 'red', 'chocolate']
    col = 0
    for i in range(len(x)):
        j=jominy(x[i])
        plt.plot(tj[:len(j)], j, label='x = ' + str(np.round(x[i], 2)) + 'mm', color=colors[col])
        col +=1
    if valg == 'Yes':
        colors1 = ['crimson', 'red', 'orange', 'chartreuse', 'limegreen', 'turquoise', 'cyan', 'blue','purple', 'hotpink']
        col1 = 0
        for i in range(len(X)):
            t1 = t(X[i])
            plt.plot(t1, T1, label='X =' + str(np.round(X[i], 2)), linestyle='--', color=colors1[col1])
            col1 +=1
    plt.grid()
    plt.title('Jominy-stick')
    plt.xscale('log')
    plt.xlabel('log t')
    plt.ylabel('Temperature, K')
    plt.xlim(0.1, 10**5)
    plt.legend()
    plt.show()

## Extrusion

The following equation was used to construct the T-t profile for extrusion:

$$ \frac{T-T_{0}}{T_{i}-T_{0}} = exp(-\frac{2h}{w\rho_{c}}t) $$

where h is the heat transfer coefficient, $\rho_{c}$ is the specific heat capacity of the material per unit volume and w is the wall thickness of the profile.

In [107]:
#constants
rhoc = 0.0027*10**(9) #J/m3C
w = 10*10**(-3) #m
h = np.array([10, 100, 10000])


In [123]:
def extrusion(h):
    T = np.zeros(len(tj))
    for i in range(len(tj)):
        T2 = T0 + (Ti-T0)*np.exp((-(2*h)/(w*rhoc))*tj[i])
        if T2 > 308:
            T[i] = T2
    return np.trim_zeros(T)

def eplot(valg):
    colors = ['steelblue','darkcyan','deepskyblue']
    labels = ['Air','Fan', 'Water']
    col = 0
    for i in range(len(h)):
        e = extrusion(h[i])
        plt.plot(tj[:len(e)], e, label=str(labels[i])+ ': ' + str(np.round(h[i], 2))+'W/m^2C', color=colors[col])
        col +=1
    if valg == 'Yes':
        colors1 = ['crimson', 'red', 'orange', 'chartreuse', 'limegreen', 'turquoise', 'cyan', 'blue','purple', 'hotpink']
        col1 = 0
        for i in range(len(X)):
            t1 = t(X[i])
            plt.plot(t1, T1, label='X =' + str(np.round(X[i], 2)), linestyle='--', color=colors1[col1])
            col1 +=1
    plt.grid()
    plt.title('Extrusion')
    plt.xscale('log')
    plt.xlabel('log t')
    plt.ylabel('Temperature, K')
    plt.xlim(0.1, 10**5)
    plt.legend()
    plt.show()


## Welding

The following equation was used to construct the T-t profile for welding:

$$ T-T_{0} = \frac{q_{0}/vd}{\rho_{c}\sqrt{4\pi at}} exp(-\frac{y^{2}}{4at}) $$

where $q_{0}$ is the net effect added through welding, v is the welding speed, d is the plate thickness and y is the distance in the transversal direction of the plate measured from the centre of the weld. 

In [120]:
#constants
q0vd = np.array([0.05*10**9, 0.1*10**9, 0.2*10**9])
a2 = 62*10**(-6) #m2/s

In [128]:
tw = np.logspace(-4, 4, 500)

def welding(q0vd1, y):
    T = np.zeros(len(tw))
    for i in range(len(tw)):
        dTdt = -(q0vd1*(2*a2*tw[i]-y**2)*np.exp(-y**2/(4*a2*tw[i])))/(8*np.sqrt(np.pi)*rhoc*tw[i]*(a2*tw[i])**(3/2))
        T2 = T0 + (q0vd1/(rhoc*np.sqrt(4*np.pi*a2*tw[i])))*np.exp(-y**2/(4*a2*tw[i]))
        if T2 > 308 or dTdt > 0:
            T[i] = T2
    return np.trim_zeros(T)


def wplot(y):
    colors = ['black','grey','deepskyblue']
    labels = ['Laser','Arc', 'High energy plasma']
    col = 0
    for i in range(len(q0vd)):
        plt.plot(tw[:len(welding(q0vd[i], y))], welding(q0vd[i], y),label=str(labels[i]) + ': ' +  str( np.round(q0vd[i]/10**8, 1))+ ' x 1e8', color=colors[col])
        col +=1
    plt.grid()
    plt.title('Welding')
    plt.xlabel('log t')
    plt.xscale('log')
    plt.ylabel('Temperature, K')
    plt.legend(title='q0/vd [W/m^2C]:')
    plt.show()

## Interactive menu

In [129]:
valgknapp = widgets.RadioButtons(options=[('Yes'), ('No')], value = 'No', description = 'C-curves?', layout={'width': 'max-content'})
Yslider = widgets.FloatSlider(value = 0.001, min = 0.001, max = 0.1, step = 0.001, description='y [m]', continuous_update=False)


color_picker = widgets.ColorPicker(description='Pick a color', value= "purple")

def sub_task_chooser(sub):
    
    # Switch function
    if sub == 1: #plotting for C-curves
        widgets.interact(Cplot)          

    elif sub == 2: #plotting for Jominy
        widgets.interact(jplot, valg=valgknapp)
        
    elif sub == 3: #plotting for extrusion
        widgets.interact(eplot, valg=valgknapp)
        
    elif sub == 4: #plotting for welding
        print("y is distance in the transversal direction of the plate measured from the centre of the weld:")
        widgets.interact(wplot, y=Yslider)
        
        

# Menu toggle buttons
sub_menu = widgets.ToggleButtons(options={'C-curves':1, 'Jominy':2, 'Extrusion':3, 'Welding':4}, value = 1, description='Plot:', button_style='danger',)
#red buttons

interact(sub_task_chooser, sub=sub_menu)

interactive(children=(ToggleButtons(button_style='danger', description='Plot:', options={'C-curves': 1, 'Jomin…

<function __main__.sub_task_chooser(sub)>